# Logistic regression using tensorflow - quick tutorial using the Machine learning code template

In [5]:
import tensorflow as tf
import os
import numpy as np

# Below are the function for the supervised learning workflow

In [6]:
# same params and variables initialization as log reg.
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")

In [7]:
# former inference is now used for combining inputs
def combine_inputs(X):
    return tf.matmul(X, W) + b

# new inferred value is the sigmoid applied to the former
def inference(X):
    return tf.sigmoid(combine_inputs(X))

In [8]:
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits= combine_inputs(X), labels=Y))

In [10]:
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)

In [11]:
def inputs():
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        read_csv(100, "train.csv", [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])

    # convert categorical data
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))

    gender = tf.to_float(tf.equal(sex, ["female"]))

    # Finally we pack all the features in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])

    return features, survived


In [19]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [13]:
def evaluate(sess, X, Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)

    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))

# Last, the run function for the model

In [20]:
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:

    tf.initialize_all_variables().run()

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])

    evaluate(sess, X, Y)

    import time
    time.sleep(5)

    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [0.6468803]
loss:  [0.68095696]
loss:  [0.76343435]
loss:  [0.77327764]
loss:  [0.6716818]
loss:  [0.7581498]
loss:  [0.66627204]
loss:  [0.6855507]
loss:  [0.67479444]
loss:  [0.70050454]
loss:  [0.65103865]
loss:  [0.67365915]
loss:  [0.6727474]
loss:  [0.63868123]
loss:  [0.7181519]
loss:  [0.63994426]
loss:  [0.64164895]
loss:  [0.59814954]
loss:  [0.64307886]
loss:  [0.7747194]
loss:  [0.6025498]
loss:  [0.58160543]
loss:  [0.63663167]
loss:  [0.6257614]
loss:  [0.6397402]
loss:  [0.61464375]
loss:  [0.6274092]
loss:  [0.6410486]
loss:  [0.63349056]
loss:  [0.6698008]
loss:  [0.5861319]
loss:  [0.62052405]
loss:  [0.63032115]
loss:  [0.65274423]
loss:  [0.57418996]
loss:  [0.60614693]
loss:  [0.6586709]
loss:  [0.7279992]
loss:  [0.5755184]
loss:  [0.5894689]
loss:  [0.6379256]
loss:  [0.6021888]
loss:  [0.6434579]
loss:  [0.5937062]
loss:  [0.5873648]
loss:  [0.61078656]
loss:  [0.6041691]
loss:  [0.6214145]
loss:  [0.59741575]
loss:  [0.59969574]
loss:  [0.6118614]
loss: 